# Adding Back the Tides
An effort to make the daily files more accurate as they are currently lacking the tidal pumping that is so important to the flow of the Salish Sea

In [1]:
import xarray as xr
from pathlib import Path
import numpy as np
import datetime as dt

In [2]:
start = dt.datetime(2019,2,28)

In [4]:

numdays=3
date_list = [start + dt.timedelta(days=x) for x in range(numdays)]

In [5]:
date_list[-1] #this should be the first day for your next run

datetime.datetime(2019, 3, 2, 0, 0)

In [6]:
path = Path("/results/SalishSea/nowcast-green.201812/")

drop_vars = (
    "bounds_lon", "bounds_lat", "area", "deptht_bounds", "PAR",
    "time_centered", "time_centered_bounds", "time_counter_bounds", "dissolved_oxygen",
    "sigma_theta", "Fraser_tracer", "dissolved_inorganic_carbon", "total_alkalnity",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1h_*_carp_T.nc")) for day in date_list]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
e3t = mydata['e3t']

In [7]:
# convert e3t to e3u, e3v, and to e3w
e3t_xshift = e3t.shift(x=-1,fill_value=0)
e3u = e3t_xshift+e3t
e3u = e3u*0.5
e3u = e3u.rename({'deptht': 'depthu'})

e3t_yshift = e3t.shift(y=-1,fill_value=0)
e3v = e3t_yshift+e3t
e3v = e3v*0.5
e3v = e3v.rename({'deptht': 'depthv'})

In [19]:
drop_vars = (
    "nav_lon", "bounds_lon", "nav_lat", "bounds_lat", "area", "depthu_bounds", 
    "time_centered", "time_centered_bounds", "time_counter_bounds",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1h_*_grid_U.nc")) for day in date_list]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
u = mydata['vozocrtx']

In [20]:
drop_vars = (
    "nav_lon", "bounds_lon", "nav_lat", "bounds_lat", "area", "depthv_bounds", 
    "time_centered", "time_centered_bounds", "time_counter_bounds",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1h_*_grid_V.nc")) for day in date_list]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
v = mydata['vomecrty']

In [46]:
xmesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask201702.nc')
umask = (xmesh.umask).rename({'z': 'depthu','t': 'time_counter'})[0,:,:,:]
vmask = (xmesh.vmask).rename({'z': 'depthv','t': 'time_counter'})[0,:,:,:]

In [48]:
#calcuate bartropic component of u
ut_h = (u*e3u*umask).sum(dim='depthu')/(e3u*umask).sum(dim='depthu')

In [49]:
#calcuate bartropic component of v
vt_h = (v*e3v*vmask).sum(dim='depthv')/(e3v*vmask).sum(dim='depthv')

In [50]:
# Now get the required data from the daily files
drop_vars = (
    "nav_lon", "bounds_lon", "nav_lat", "bounds_lat", "area", "deptht_bounds", "PAR",
    "time_centered", "time_centered_bounds", "time_counter_bounds", "dissolved_oxygen",
    "sigma_theta", "Fraser_tracer", "dissolved_inorganic_carbon", "total_alkalnity",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1d_*_carp_T.nc")) for day in date_list]
# files = [sorted(path.glob("{:02}mar19/SalishSea_1d_*_carp_T.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
e3t_d = mydata['e3t']

drop_vars = (
    "nav_lon", "bounds_lon", "nav_lat", "bounds_lat", "area", "depthu_bounds", 
    "time_centered", "time_centered_bounds", "time_counter_bounds",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1d_*_grid_U.nc")) for day in date_list]
# files = [sorted(path.glob("{:02}mar19/SalishSea_1d_*_grid_U.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
u_d = mydata['vozocrtx']

drop_vars = (
    "nav_lon", "bounds_lon", "nav_lat", "bounds_lat", "area", "depthv_bounds", 
    "time_centered", "time_centered_bounds", "time_counter_bounds",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1d_*_grid_V.nc")) for day in date_list]
# files = [sorted(path.glob("{:02}mar19/SalishSea_1d_*_grid_V.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
v_d = mydata['vomecrty']

In [51]:
# convert e3t to e3u and to e3v
e3t_xshift = e3t_d.shift(x=-1,fill_value=0)
e3u_d = e3t_xshift+e3t_d
e3u_d = e3u_d*0.5
e3u_d = e3u_d.rename({'deptht': 'depthu'})

e3t_yshift = e3t_d.shift(y=-1,fill_value=0)
e3v_d = e3t_yshift+e3t_d
e3v_d = e3v_d*0.5
e3v_d = e3v_d.rename({'deptht': 'depthv'})

In [52]:
#calcuate bartropic component
ut_d = (u_d*e3u_d*umask).sum(dim='depthu')/(e3u_d*umask).sum(dim='depthu')

In [53]:
#subtract from u to get baroclinic component
uc_d = u_d-ut_d #does this work even though their ut_d lacks the depth dimension?

interpolate + resample uc_d to get it in an hourly format

In [54]:
uc_h_interp = uc_d.resample(time_counter="1H", loffset="30min").interpolate("linear")

In [59]:
u_new = ut_h  + uc_h_interp

In [60]:
u_new = u_new.isel(time_counter = np.arange(12,36,1))

/home/rbeutel/.local/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [61]:
u_new = u_new*umask

In [65]:
u_new = u_new.transpose('time_counter','depthu','y','x')
u_new

<xarray.DataArray (time_counter: 24, depthu: 40, y: 898, x: 398)>
dask.array<transpose, shape=(24, 40, 898, 398), dtype=float32, chunksize=(24, 40, 898, 398), chunktype=numpy.ndarray>
Coordinates:
  * depthu        (depthu) float64 0.5 1.5 2.5 3.5 ... 360.7 387.6 414.5 441.5
  * time_counter  (time_counter) datetime64[ns] 2019-03-01T00:30:00 ... 2019-...
    nav_lat       (y, x) float32 dask.array<chunksize=(898, 398), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(898, 398), meta=np.ndarray>
Dimensions without coordinates: y, x

In [53]:
encoding={
          "vozocrtx": {"zlib": True, "complevel": 4, "_FillValue": np.nan}
}

In [ ]:
path = "/data/rbeutel/analysis/ssc_tidesback/"
print('u_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list_hourly[0],date_list_hourly[0]))
u_new.to_netcdf(str(path)+'u_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[0]), encoding=encoding)

In [ ]:
u_new.to_netcdf(str(path)+'u_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))

In [ ]:
#calcuate bartropic component
vt_d = (v*e3v).sum(dim='depthv')/e3v.sum(dim='depthv')

In [ ]:
#subtract from v to get baroclinic component
vc_d = v-vt_d 

In [ ]:
vc_h_interp = vc_d.resample(time_counter="1H", loffset="30min").interpolate("linear")

In [ ]:
v_new = vt_h  + vc_h_interp

In [ ]:
v_new.to_netcdf(str(path)+'v_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))